In [ ]:
import contextlib
import json

from capella import lee_filter

from matplotlib import rcParams
from matplotlib import pyplot as plt

import numpy as np
import rasterio
from rasterio.merge import merge
from rasterio.io import MemoryFile
from rasterio.plot import show
from rasterio.warp import transform_bounds
from rasterio import windows
from rasterio.crs import CRS
from rasterio.plot import reshape_as_image
from skimage import exposure
from skimage.restoration import estimate_sigma
from scipy.ndimage import morphology
from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance

# Allow division by zero
np.seterr(divide='ignore', invalid='ignore')

%matplotlib inline

In [ ]:
with open('filter.json') as f:
    filters = json.load(f)
    BBOX = filters['bbox']

In [ ]:
result = ! rio capella --credentials credentials.json --area filter.json --collection rotterdam-aerial --limit 50 query
fc = json.loads(result[0])
features = fc['features']

In [ ]:
@contextlib.contextmanager
def make_inputs(features):
    datasets = []
    for ft in features:
        fid = f"tiledb://capellaspace/{ft['collection']}_{ft['id']}"
        datasets.append(rasterio.open(fid))

    yield datasets
    
    for ds in datasets:
        ds.close()

In [ ]:
# uses an additional band as a counter
def myaverage(old_data, new_data, old_nodata, new_nodata, index, roff, coff):
    mask = ~new_nodata[0]
    old_data[0, mask] += new_data[0][mask]
    old_data[1, mask] += 1

with make_inputs(features) as datasets:
    result, _ = merge(datasets, transform_bounds(CRS.from_epsg(4326), datasets[0].crs, *BBOX),
                      nodata=0, output_count=2, dtype=np.uint64, method=myaverage)

av = result[0] / result[1]
equ = exposure.adjust_log(av, gain=10)

rcParams['figure.figsize'] = 10, 10
fig, ax = plt.subplots()
ax.imshow(equ, cmap='gray')
ax.set_title("Counter method - Mean Image")

In [ ]:
# stacks inputs and then computes mean across the bands
# this is an alternative and flexible way to compute the same result as above
def stack_average(old_data, new_data, old_nodata, new_nodata, index, roff, coff):
    mask = ~new_nodata[0]
    old_data[index, mask] = new_data[0][mask]
    
with make_inputs(features) as datasets:
    output_count = len(datasets) 
    result, _ = merge(datasets, transform_bounds(CRS.from_epsg(4326), datasets[0].crs, *BBOX),
                      nodata=0, output_count=output_count, method=stack_average)

av = result.sum(axis=0) / (result != 0).sum(axis=0)        
equ = exposure.adjust_log(av, gain=10)

rcParams['figure.figsize'] = 10, 10
fig, ax = plt.subplots()
ax.imshow(equ, cmap='gray')
ax.set_title("Stack method - Mean Image")

In [ ]:
# SNR weighted average
with make_inputs(features) as datasets:
    output_count = len(datasets) 
    stack, _ = merge(datasets, transform_bounds(CRS.from_epsg(4326), datasets[0].crs, *BBOX),
                      nodata=0, output_count=output_count, method=stack_average)

# estimate the average noise standard deviation across layers in the stack, could use a local window and calculate variance instead of this built-in function
img = reshape_as_image(stack)
sigmas = np.array(estimate_sigma(img, multichannel=True))

# Pixel SNR is the ratio of the local mean to the noise standard deviation for that image in the stack 
weights = img / sigmas

result = (img * weights).sum(axis=2) / weights.sum()
equ = exposure.adjust_log(av, gain=10)

rcParams['figure.figsize'] = 10, 10
fig, ax = plt.subplots()
ax.imshow(equ, cmap='gray')
ax.set_title("Weighted Image")